## Imports

In [1]:
import pandas as pd
import numpy as np
from random import random

## Read data

In [2]:
osago_sales_rejected_trafic_df = pd.DataFrame()

In [3]:
osago_sales_premium_by_partner_df  = pd.DataFrame()

In [4]:
osago_sales_non_prem_by_partner_df = pd.DataFrame()

In [5]:
osago_sum_sales_df = pd.DataFrame() #есть два отдельных графика, но можно взять один

In [6]:
osago_sales_by_segment_df = pd.DataFrame()

In [7]:
credits_requests_sum_df = pd.DataFrame()

In [8]:
credits_requests_sum_by_segment_df = pd.DataFrame()

In [9]:
credits_requests_by_partner_df = pd.DataFrame()

In [10]:
mfo_conversion_rate_by_partner = pd.DataFrame()

In [11]:
mfo_clicks_by_partners_df = pd.read_excel('test.xlsx', sheet_name = 'Real Data', index_col = 0)
#mfo_clicks_df.head()

In [12]:
mfo_clicks_by_offer_df = pd.DataFrame()

In [13]:
mfo_releases_by_partners_df = pd.read_excel(
    'mfo_releases_by_partners.xlsx', 
    #sheet_name = 'Real Data', 
    index_col = 0)
#mfo_releases_by_partners_df

In [14]:
mfo_releases_by_offer_df = pd.DataFrame()

In [15]:
mfp_negative_income_df = pd.DataFrame()

## Functions

### Appear / diappear

#### get_days_buying (non zero days until yesterday)

In [16]:
def days_by_row(r):
    df = pd.DataFrame(r)
    df = df.rename(columns = {df.columns[0]: 'c'})
    df = df.iloc[::-1].reset_index(drop = True)
    df['row_1_zero_index'] = (df.index + 1) * (df['c'] == 0)
    days_buying = df.loc[df['row_1_zero_index'] > 0, 'row_1_zero_index'].min() - 1
    if np.isnan(days_buying):
        days_buying = df.shape[0]        
    return days_buying

In [17]:
def get_days_buying(df):
    tmp = pd.DataFrame()
    for r in df.index:
        tmp.loc[r, 'days_buying'] = days_by_row(df.loc[r])
    return tmp

In [18]:
#mfo_clicks_by_partners_df

In [19]:
get_days_buying(mfo_clicks_by_partners_df)

,days_buying
"6926, p1sms.ru",91.0
"7095, MVP_Group",91.0
"7103, Онлайн Деньги.ру",91.0
"7086, ZaimBesplatn",91.0
"4674, Лидстех",91.0
...,...
"2968, Александр Воробьев",0.0
"2758, Марсель Имаев",0.0
"1625, Тестовый аккаунт",0.0
"123, e-osago.sait_mailforspam.com",0.0


#### get_zero_days by row

In [20]:
def zero_days_by_row(r):
    df = pd.DataFrame(r)
    df = df.rename(columns = {df.columns[0]: 'c'})
    df = df.iloc[::-1].reset_index(drop = True)
    df['row_zero_index'] = (df.index + 1) * (df['c'] != 0)
    res = df.loc[df['row_zero_index'] != 0, 'row_zero_index'].min() - 1
    if np.isnan(res):
        res = df.shape[0]        
        
    return res

#zero_days_by_row(mfo_clicks_df.iloc[0])

In [21]:
def get_zero_days(df):
    tmp = pd.DataFrame()
    for r in df.index:
        tmp.loc[r, 'zero_days'] = zero_days_by_row(df.loc[r])
    return tmp

#get_zero_days(mfo_clicks_df)

#### get_zero_days_before_yesterday

In [22]:
def zero_days_before_yesterday_by_row(r):
    df = pd.DataFrame(r)
    df = df.rename(columns = {df.columns[0]: 'c'})
    df = df.iloc[::-1].iloc[1:, :].reset_index(drop = True)
    df['row_zero_index'] = (df.index + 1) * (df['c'] != 0)
    res = df.loc[df['row_zero_index'] != 0, 'row_zero_index'].min() - 1
    if np.isnan(res):
        res = df.shape[0]        
        
    return res

#zero_days_before_yesterday_by_row(mfo_clicks_df.iloc[0])

In [23]:
def get_zero_days_before_yesterday(df):
    tmp = pd.DataFrame()
    for r in df.index:
        tmp.loc[r, 'zero_days_before_yesterday'] = zero_days_before_yesterday_by_row(df.loc[r])
    return tmp

#get_zero_days_before_yesterday(mfo_clicks_df)

#### get_days_buying_before_yesterday_by_row

In [24]:
def days_buying_before_yesterday_by_row(r):
    df = pd.DataFrame(r)
    df = df.rename(columns = {df.columns[0]: 'c'})
    df = df.iloc[::-1]
    df = df.iloc[1:, :].reset_index(drop = True)
    df['row_1_zero_index'] = (df.index + 1) * (df['c'] == 0)
    days_buying = df.loc[df['row_1_zero_index'] > 0, 'row_1_zero_index'].min() - 1
    if np.isnan(days_buying):
        days_buying = df.shape[0]        
    return days_buying

In [25]:
def get_days_buying_before_yesterday_by_row(df):
    tmp = pd.DataFrame()
    for r in df.index:
        tmp.loc[r, 'days_buying_before_yesterday'] = days_buying_before_yesterday_by_row(df.loc[r])
    return tmp

In [26]:
#get_days_buying_before_yesterday_by_row(mfo_clicks_df)

#### is zero yesterday

In [33]:
def is_zero_yesterday(df):
    tmp = pd.DataFrame()
    tmp['is_zero_yesterday'] = df.iloc[:, -1] == 0
    return tmp['is_zero_yesterday']

In [34]:
#mfo_clicks_df

In [35]:
#is_zero_yesterday(mfo_clicks_df)

### Reference values


#### get_avg_by_row

In [27]:
def avg_by_row(r):
    df = pd.DataFrame(r)
    df = df.rename(columns = {df.columns[0]: 'c'})
    avg = df['c'].mean()
    return avg

#avg_by_row(mfo_clicks_df.iloc[1, -5:-1])

In [28]:
def get_avg_by_row(df):
    #display(df)
    tmp = pd.DataFrame()
    for r in df.index:
        tmp.loc[r, 'avg_by_row'] = avg_by_row(df.loc[r].iloc[:-1])
    return tmp

#get_avg_by_row(mfo_clicks_df)

#### get_median_by_row

In [78]:
def median_by_row(r):
    df = pd.DataFrame(r)
    df = df.rename(columns = {df.columns[0]: 'c'})
    out = df['c'].median()
    return out

median_by_row(mfo_clicks_by_partners_df.iloc[1, -6:-1])

8207.0

In [30]:
def get_median_by_row(df):
    #display(df)
    tmp = pd.DataFrame()
    for r in df.index:
        tmp.loc[r, 'avg_by_row'] = avg_by_row(df.loc[r].iloc[:-1])
    return tmp

get_median_by_row(mfo_clicks_by_partners_df)

,avg_by_row
"6926, p1sms.ru",8815.388889
"7095, MVP_Group",9015.922222
"7103, Онлайн Деньги.ру",2436.022222
"7086, ZaimBesplatn",2521.233333
"4674, Лидстех",1861.633333
...,...
"2968, Александр Воробьев",2.488889
"2758, Марсель Имаев",0.177778
"1625, Тестовый аккаунт",0.266667
"123, e-osago.sait_mailforspam.com",0.011111


#### get_avg_by_non_zero_by_row

In [31]:
def avg_by_non_zero_by_row(r):
    df = pd.DataFrame(r)
    df = df.rename(columns = {df.columns[0]: 'c'}).iloc[::-1].iloc[1:, :]
    avg = df.loc[df['c']>0, 'c'].mean()
    if np.isnan(avg):
        avg = 0
    return avg
#avg_by_non_zero_by_row(mfo_clicks_df.iloc[0])

In [32]:
def get_avg_by_non_zero_by_row(df):
    tmp = pd.DataFrame()
    for r in df.index:
        tmp.loc[r, 'avg_by_nonzero_row'] = avg_by_non_zero_by_row(df.loc[r])
    return tmp

#get_avg_by_non_zero_by_row(mfo_clicks_df)

#### share + numbers_of_days_buying_before_yesterday 

In [77]:
def stat_of_days_buying_before_yesterday(df):
    n_rows = df.shape[0]
    n_cols = df.shape[1]
    #print('n_cols=', n_cols)
    #print('n_rows=', n_rows)
    tmp = df > 0
    tmp['share_non_zero_days']     = tmp.iloc[:, :n_cols+1].mean(axis=1)
    tmp['number_of_non_zero_days'] = tmp.iloc[:, :n_cols+0].sum(axis=1)
    tmp['number_of_days']          = tmp.iloc[:, :n_cols-0].count(axis=1)
    
    tmp['zero_days_stat'] = (
        '['
        +tmp['number_of_non_zero_days'].astype('str')
        +'/'
        +tmp['number_of_days'].astype('str')
        +']'
    )
    
    return tmp[[
        'share_non_zero_days', 
    #    'number_of_days', 
    #    'number_of_non_zero_days', 
        'zero_days_stat'
    ]]
    
stat_of_days_buying_before_yesterday(mfo_clicks_by_partners_df)

,share_non_zero_days,zero_days_stat
partner,,
"6926, p1sms.ru",1.000000,[91/91]
"7095, MVP_Group",1.000000,[91/91]
"7103, Онлайн Деньги.ру",1.000000,[91/91]
"7086, ZaimBesplatn",1.000000,[91/91]
"4674, Лидстех",1.000000,[91/91]
...,...,...
"2968, Александр Воробьев",0.725275,[66/91]
"2758, Марсель Имаев",0.065934,[6/91]
"1625, Тестовый аккаунт",0.142857,[13/91]


#### Increase / decrease rate.

In [40]:
def increase_rate_by_row(r, n_input = 999_999_999):
    '''
    r - row with values (per partner, offers, ...)
    n - number of days to calculate mean. 999_999_999 by default - to aggragate whole row 
    '''
    n_max = len(r) - 1 #real count of values in row
    n = min(n_input, n_max) #number of rows to calculate mean

    df = pd.DataFrame(r)
    current_value = df.iloc[-1, 0] #get last value in row, usually, this is yesterdays value
    prev_avg_value = avg_by_row(r[-n-1:-1])
    #print('curr_value     =', current_value)
    #print('prev_avg_value =', prev_avg_value)

    if (prev_avg_value == 0) & (current_value == 0):
        #print('0/0=1')
        inc_rate = 1
    elif (prev_avg_value == 0) & (current_value != 0):
        #print('1/0=0')
        inc_rate = np.inf
    elif (prev_avg_value != 0) & (current_value == 0):
        #print('0/0=1')
        inc_rate = current_value / prev_avg_value
    elif (prev_avg_value != 0) & (current_value != 0):
        #print('0/0=1')
        inc_rate = current_value / prev_avg_value
    else:
        inc_rate = 666_666_666
    
    #print(inc_rate)
    return inc_rate 

#display(t.iloc[1])
#increase_rate_by_row(mfo_clicks_df.iloc[0], 5)

In [41]:
def get_increase_rate(df, n=999_999_999):
    tmp = pd.DataFrame()
    for r in df.index:
        tmp.loc[r, 'inc_rate_'+str(n)] = increase_rate_by_row(df.loc[r], n)
    return tmp

#get_increase_rate(mfo_clicks_df, n=30)

## Prepare data for alert processing.

In [42]:
def make_data_table(df):
    out = (df
        .join(get_days_buying(df))
        .join(get_days_buying_before_yesterday_by_row(df))
        .join(get_avg_by_row(df))
        .join(get_avg_by_non_zero_by_row(df))
        .join(is_zero_yesterday(df))
        .join(get_zero_days(df))
        .join(get_zero_days_before_yesterday(df))
        .join(stat_of_days_buying_before_yesterday(df))
        .join(get_increase_rate(df, n=1))
        .join(get_increase_rate(df, n=7))
        .join(get_increase_rate(df, n=30))
    )
    return out

**Загрузка данных о кликах.**

In [49]:
mfo_clicks_by_partners_data_df = make_data_table(mfo_clicks_by_partners_df)
mfo_clicks_by_partners_data_df.head()

,2023-08-13,2023-08-14,2023-08-15,2023-08-16,2023-08-17,2023-08-18,2023-08-19,2023-08-20,2023-08-21,2023-08-22,...,avg_by_row,avg_by_nonzero_row,is_zero_yesterday,zero_days,zero_days_before_yesterday,share_non_zero_days,zero_days_stat,inc_rate_1,inc_rate_7,inc_rate_30
partner,,,,,,,,,,,,,,,,,,,,,
"6926, p1sms.ru",3137,3282,4116,6152,4306,4041,2903,4320,4762,8070,...,8815.388889,8815.388889,False,0.0,0.0,1.0,[91/91],0.900155,0.868737,0.955421
"7095, MVP_Group",14345,15713,18751,22393,21869,22394,22324,16430,16199,16271,...,9015.922222,9015.922222,False,0.0,0.0,1.0,[91/91],0.920068,0.921415,0.961270
"7103, Онлайн Деньги.ру",2312,2318,2458,2381,2252,2301,2521,2419,2450,2660,...,2436.022222,2436.022222,False,0.0,0.0,1.0,[91/91],1.042328,1.399188,1.091686
"7086, ZaimBesplatn",1371,1595,1565,2362,3015,2778,2898,2848,2605,2752,...,2521.233333,2521.233333,False,0.0,0.0,1.0,[91/91],1.017657,1.127155,0.857620
"4674, Лидстех",1852,1591,1608,1522,2270,1648,1697,1663,1652,1751,...,1861.633333,1861.633333,False,0.0,0.0,1.0,[91/91],1.794454,1.646355,1.610188


In [47]:
mfo_releases_by_partners_data_df = make_data_table(mfo_releases_by_partners_df)
mfo_releases_by_partners_data_df.head()

,2023-08-22 00:00:00,2023-08-23 00:00:00,2023-08-24 00:00:00,2023-08-25 00:00:00,2023-08-26 00:00:00,2023-08-27 00:00:00,2023-08-28 00:00:00,2023-08-29 00:00:00,2023-08-30 00:00:00,2023-08-31 00:00:00,...,avg_by_row,avg_by_nonzero_row,is_zero_yesterday,zero_days,zero_days_before_yesterday,share_non_zero_days,zero_days_stat,inc_rate_1,inc_rate_7,inc_rate_30
partner,,,,,,,,,,,,,,,,,,,,,
"6926, p1sms.ru",118,96,103,99,127,143,169,163,189,195,...,206.033333,206.033333,False,0.0,0.0,1.000000,[91/91],1.081152,1.298742,1.455592
"7095, MVP_Group",445,423,448,447,436,479,442,256,113,68,...,172.733333,172.733333,False,0.0,0.0,1.000000,[91/91],1.324074,1.284981,0.890596
"7656, Kosmos_Leads",223,202,249,226,267,304,314,254,201,201,...,114.211111,123.843373,True,6.0,5.0,0.912088,[83/91],1.000000,0.000000,0.000000
"7086, ZaimBesplatn",75,93,103,110,92,105,131,90,85,89,...,82.977778,82.977778,False,0.0,0.0,1.000000,[91/91],1.126582,1.177694,0.997385
"8013, Teleport",87,93,71,81,76,80,68,66,109,84,...,72.588889,72.588889,False,0.0,0.0,1.000000,[91/91],1.428571,1.320755,0.896593


## Alerts

### MFO clicks

In [74]:
print('Вывожу статистику кликов партнёров, которые ранее были, а вчера исчезли:')
mfo_clicks_disappeared_alerts_df = mfo_clicks_by_partners_data_df.query('is_zero_yesterday == True & zero_days_before_yesterday == 0')

tt=mfo_clicks_by_partners_data_df

mfo_clicks_disappeared_alerts_out = pd.DataFrame()

for r in mfo_clicks_disappeared_alerts_df.index:
    
    tmp = pd.DataFrame()

    days_buying_before_yesterday = tt.loc[[r], 'days_buying_before_yesterday'][0]
    share_non_zero_days = tt.loc[[r], 'share_non_zero_days'][0]
    avg_by_nonzero_row = tt.loc[[r], 'avg_by_nonzero_row'][0]
    stat_of_days_buying_before_yesterday = tt.loc[[r], 'zero_days_stat'][0]

    alert_msg = (
        ''
        + 'Партнёр ' 
        + r 
        + ' лил трафик' 
        + str(int(days_buying_before_yesterday)) 
        + ' дней до того, как вчера пропал. За период лил трафик ' 
        + str(round(share_non_zero_days * 100, 1)) 
        + '% от периода '
        + stat_of_days_buying_before_yesterday
        + ' в среднем по ' 
        + str(round(avg_by_nonzero_row, 1)) 
        + ' за день.'
    )

    tmp.loc[0, 'Продукт']   = 'МФО'
    tmp.loc[0, 'Метрика']   = 'Исчезли клики'
    tmp.loc[0, 'Сообщение'] = alert_msg
    tmp.loc[0, 'Важность']  = random()
    
    mfo_clicks_disappeared_alerts_out = pd.concat([mfo_clicks_disappeared_alerts_out, tmp])
    
    print(alert_msg)

mfo_clicks_disappeared_alerts_out.reset_index(drop = True)

Вывожу статистику кликов партнёров, которые ранее были, а вчера исчезли:
Партнёр 6388, Yoomwi.site лил трафик1 дней до того, как вчера пропал. За период лил трафик 62.6% от периода [57/91] в среднем по 52.3 за день.
Партнёр 2968, Александр Воробьев лил трафик7 дней до того, как вчера пропал. За период лил трафик 72.5% от периода [66/91] в среднем по 3.4 за день.
Партнёр 2758, Марсель Имаев лил трафик2 дней до того, как вчера пропал. За период лил трафик 6.6% от периода [6/91] в среднем по 2.7 за день.
Партнёр 1625, Тестовый аккаунт лил трафик1 дней до того, как вчера пропал. За период лил трафик 14.3% от периода [13/91] в среднем по 1.8 за день.
Партнёр 123, e-osago.sait_mailforspam.com лил трафик1 дней до того, как вчера пропал. За период лил трафик 1.1% от периода [1/91] в среднем по 1.0 за день.
Партнёр 5156, Игорь Рыбалкин лил трафик5 дней до того, как вчера пропал. За период лил трафик 82.4% от периода [75/91] в среднем по 15.8 за день.


,Продукт,Метрика,Сообщение,Важность
0,МФО,Исчезли клики,"Партнёр 6388, Yoomwi.site лил трафик1 дней до ...",0.416766
1,МФО,Исчезли клики,"Партнёр 2968, Александр Воробьев лил трафик7 д...",0.690758
2,МФО,Исчезли клики,"Партнёр 2758, Марсель Имаев лил трафик2 дней д...",0.328081
3,МФО,Исчезли клики,"Партнёр 1625, Тестовый аккаунт лил трафик1 дне...",0.045002
4,МФО,Исчезли клики,"Партнёр 123, e-osago.sait_mailforspam.com лил ...",0.994350
5,МФО,Исчезли клики,"Партнёр 5156, Игорь Рыбалкин лил трафик5 дней ...",0.436395


### MFO release

In [65]:
mfo_releases_by_partners_data_df.head()

,2023-08-22 00:00:00,2023-08-23 00:00:00,2023-08-24 00:00:00,2023-08-25 00:00:00,2023-08-26 00:00:00,2023-08-27 00:00:00,2023-08-28 00:00:00,2023-08-29 00:00:00,2023-08-30 00:00:00,2023-08-31 00:00:00,...,avg_by_row,avg_by_nonzero_row,is_zero_yesterday,zero_days,zero_days_before_yesterday,share_non_zero_days,zero_days_stat,inc_rate_1,inc_rate_7,inc_rate_30
partner,,,,,,,,,,,,,,,,,,,,,
"6926, p1sms.ru",118,96,103,99,127,143,169,163,189,195,...,206.033333,206.033333,False,0.0,0.0,1.000000,[91/91],1.081152,1.298742,1.455592
"7095, MVP_Group",445,423,448,447,436,479,442,256,113,68,...,172.733333,172.733333,False,0.0,0.0,1.000000,[91/91],1.324074,1.284981,0.890596
"7656, Kosmos_Leads",223,202,249,226,267,304,314,254,201,201,...,114.211111,123.843373,True,6.0,5.0,0.912088,[83/91],1.000000,0.000000,0.000000
"7086, ZaimBesplatn",75,93,103,110,92,105,131,90,85,89,...,82.977778,82.977778,False,0.0,0.0,1.000000,[91/91],1.126582,1.177694,0.997385
"8013, Teleport",87,93,71,81,76,80,68,66,109,84,...,72.588889,72.588889,False,0.0,0.0,1.000000,[91/91],1.428571,1.320755,0.896593


In [73]:
print('Вывожу статистику выдач по партнёрам, которые ранее были, а вчера исчезли:')
mfo_release_alerts_df = mfo_releases_by_partners_data_df.query(
    'is_zero_yesterday == True & zero_days_before_yesterday == 0')

tt = mfo_releases_by_partners_data_df

mfo_release_disappeared_alerts_out = pd.DataFrame()

for r in mfo_release_alerts_df.index:
    
    tmp = pd.DataFrame()

    days_buying_before_yesterday = tt.loc[[r], 'days_buying_before_yesterday'][0]
    share_non_zero_days = tt.loc[[r], 'share_non_zero_days'][0]
    avg_by_nonzero_row = tt.loc[[r], 'avg_by_nonzero_row'][0]
    stat_of_days_buying_before_yesterday = tt.loc[[r], 'zero_days_stat'][0]

    alert_msg = (
        ''
        + 'По партнёру "' 
        + r 
        + '" прекратились выдачи, до этого выдачи были ' 
        + str(int(days_buying_before_yesterday)) 
        + ' дней подряд. За период выдачи были ' 
        + str(round(share_non_zero_days * 100, 1)) 
        + '% дней '
        + stat_of_days_buying_before_yesterday
        + ' в среднем по ' 
        + str(round(avg_by_nonzero_row, 1)) 
        + ' в день.'
    )

    tmp.loc[0, 'Продукт']   = 'МФО'
    tmp.loc[0, 'Метрика']   = 'Исчезли выдачи'
    tmp.loc[0, 'Сообщение'] = alert_msg
    tmp.loc[0, 'Важность']  = random()
    
    mfo_release_disappeared_alerts_out = pd.concat([mfo_release_disappeared_alerts_out, tmp])
    
    print(alert_msg)

print('Job done!')

mfo_release_alerts.reset_index(drop = True)

Вывожу статистику выдач по партнёрам, которые ранее были, а вчера исчезли:
По партнёру "7209, Rocket_Smm" прекратились выдачи, до этого выдачи были 1 дней подряд. За период выдачи были 71.4% дней [65/91] в среднем по 9.5 в день.
По партнёру "8015, Турбозайм_Контекст" прекратились выдачи, до этого выдачи были 2 дней подряд. За период выдачи были 42.9% дней [39/91] в среднем по 4.3 в день.
По партнёру "3431, Далид" прекратились выдачи, до этого выдачи были 7 дней подряд. За период выдачи были 80.2% дней [73/91] в среднем по 2.2 в день.
По партнёру "8043, Social market" прекратились выдачи, до этого выдачи были 13 дней подряд. За период выдачи были 58.2% дней [53/91] в среднем по 1.8 в день.
По партнёру "9234, artur_osaulenko" прекратились выдачи, до этого выдачи были 1 дней подряд. За период выдачи были 2.2% дней [2/91] в среднем по 2.0 в день.
Job done!


,Продукт,Метрика,Сообщение,Важность
0,МФО,Количество выдач,"По партнёру ""7209, Rocket_Smm"" прекратились вы...",0.432539
1,МФО,Количество выдач,"По партнёру ""8015, Турбозайм_Контекст"" прекрат...",0.657219
2,МФО,Количество выдач,"По партнёру ""3431, Далид"" прекратились выдачи,...",0.524900
3,МФО,Количество выдач,"По партнёру ""8043, Social market"" прекратились...",0.548257
4,МФО,Количество выдач,"По партнёру ""9234, artur_osaulenko"" прекратили...",0.366044


# Final alerts list

In [75]:
final_alerts_list = pd.concat(
    [
        mfo_clicks_disappeared_alerts_out
        , mfo_release_disappeared_alerts_out
    ]
).sort_values(by = 'Важность', ascending = False)

NUMBER_OF_ALERTS_TO_SHOW = 10

final_alerts_list.head(NUMBER_OF_ALERTS_TO_SHOW)

,Продукт,Метрика,Сообщение,Важность
0,МФО,Исчезли клики,"Партнёр 123, e-osago.sait_mailforspam.com лил ...",0.994350
0,МФО,Исчезли выдачи,"По партнёру ""8015, Турбозайм_Контекст"" прекрат...",0.832905
0,МФО,Исчезли клики,"Партнёр 2968, Александр Воробьев лил трафик7 д...",0.690758
0,МФО,Исчезли выдачи,"По партнёру ""7209, Rocket_Smm"" прекратились вы...",0.665218
0,МФО,Исчезли клики,"Партнёр 5156, Игорь Рыбалкин лил трафик5 дней ...",0.436395
0,МФО,Исчезли клики,"Партнёр 6388, Yoomwi.site лил трафик1 дней до ...",0.416766
0,МФО,Исчезли выдачи,"По партнёру ""3431, Далид"" прекратились выдачи,...",0.375268
0,МФО,Исчезли клики,"Партнёр 2758, Марсель Имаев лил трафик2 дней д...",0.328081
0,МФО,Исчезли выдачи,"По партнёру ""9234, artur_osaulenko"" прекратили...",0.281982
0,МФО,Исчезли выдачи,"По партнёру ""8043, Social market"" прекратились...",0.153118
